In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
import pandas as pd
ad = pd.read_csv("/kaggle/input/insurance/insurance.csv")
df = ad.copy()
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'female' else 0)

In [ ]:
df['smoker'] = df['smoker'].apply(lambda x: 1 if x == 'yes' else 0)

In [ ]:
df['region'] = df['region'].map({'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3})

In [ ]:
df.head()

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
sns.pairplot(df, kind  ="reg");

In [ ]:
sns.jointplot(x = "charges", y = "smoker", data = df, kind = "reg")

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X = df[["charges"]]
y = df["smoker"]
reg = LinearRegression()
model = reg.fit(X, y)
model.intercept_

In [ ]:
model.coef_

In [ ]:
model.score(X,y)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [ ]:
X = df.drop("smoker", axis = 1)
y = df["smoker"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 42)

print("X_train", X_train.shape)
print("y_train",y_train.shape)
print("X_test",X_test.shape)
print("y_test",y_test.shape)
training = df.copy()
print("training", training.shape)

In [ ]:
lm = LinearRegression()
model = lm.fit(X_train, y_train)

In [ ]:
model.intercept_

In [ ]:
model.coef_

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
rmse

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse

In [ ]:
r2_score(y_test, model.predict(X_test))

In [ ]:
X = df.copy()
X = X.drop(["smoker"], axis=1)
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

training = df.copy()
print("training", training.shape)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

pca = PCA()

In [ ]:
X_reduced_train = pca.fit_transform(scale(X_train))

In [ ]:
X_reduced_train.shape

In [ ]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals = 4)*100)[0:6]

In [ ]:
lm = LinearRegression()

In [ ]:
pcr_model = lm.fit(X_reduced_train, y_train)

In [ ]:
pcr_model.intercept_

In [ ]:
pcr_model.coef_

In [ ]:
y_pred = pcr_model.predict(X_reduced_train)

In [ ]:
y_pred[0:6]

In [ ]:
np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
r2_score(y_train, y_pred)

In [ ]:
pca2 = PCA()

In [ ]:
X_reduced_test = pca2.fit_transform(scale(X_test))

In [ ]:
y_pred = pcr_model.predict(X_reduced_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
lm = LinearRegression()
pcr_model = lm.fit(X_reduced_train[:,0:6], y_train)
y_pred = pcr_model.predict(X_reduced_test[:,0:6])
print(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn import model_selection

In [ ]:
cv_10 = model_selection.KFold(n_splits = 10,
                             shuffle = True,
                             random_state = 1)

In [ ]:
lm = LinearRegression()

In [ ]:
RMSE = []

In [ ]:
for i in np.arange(1, X_reduced_train.shape[1] + 1):
    
    score = np.sqrt(-1*model_selection.cross_val_score(lm, 
                                                       X_reduced_train[:,:i], 
                                                       y_train.ravel(), 
                                                       cv=cv_10, 
                                                       scoring='neg_mean_squared_error').mean())
    RMSE.append(score)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(RMSE, '-v')
plt.xlabel('Bileşen Sayısı')
plt.ylabel('RMSE')
plt.title('Smoker Tahmin Modeli İçin PCR Model Tuning');

In [ ]:
lm = LinearRegression()

In [ ]:
pcr_model = lm.fit(X_reduced_train[:,0:6], y_train)
y_pred = pcr_model.predict(X_reduced_train[:,0:6])
print(np.sqrt(mean_squared_error(y_train, y_pred)))

In [ ]:
y_pred = pcr_model.predict(X_reduced_test[:,0:6])

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
df.head()

In [ ]:
X = df.drop(['smoker'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.cross_decomposition import PLSRegression, PLSSVD

In [ ]:
pls_model = PLSRegression().fit(X_train, y_train)

In [ ]:
pls_model.coef_

In [ ]:
y_pred = pls_model.predict(X_train)

In [ ]:
np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
y_pred = pls_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
#CV
cv_10 = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)


#Hata hesaplamak için döngü
RMSE = []

for i in np.arange(1, X_train.shape[1] + 1):
    pls = PLSRegression(n_components=i)
    score = np.sqrt(-1*cross_val_score(pls, X_train, y_train, cv=cv_10, scoring='neg_mean_squared_error').mean())
    RMSE.append(score)

#Sonuçların Görselleştirilmesi
plt.plot(np.arange(1, X_train.shape[1] + 1), np.array(RMSE), '-v', c = "r")
plt.xlabel('Bileşen Sayısı')
plt.ylabel('RMSE')
plt.title('Smoker');

In [ ]:
pls_model = PLSRegression(n_components = 4).fit(X_train, y_train)

In [ ]:
y_pred = pls_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
df.head()

In [ ]:
X = df.drop(['smoker'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_model = Ridge(alpha = 0.1).fit(X_train, y_train)

In [ ]:
ridge_model

In [ ]:
ridge_model.coef_

In [ ]:
lambdalar = 10**np.linspace(10,-2,100)*0.5 

ridge_model = Ridge()
katsayilar = []

for i in lambdalar:
    ridge_model.set_params(alpha = i)
    ridge_model.fit(X_train, y_train) 
    katsayilar.append(ridge_model.coef_) 
    

    
ax = plt.gca()
ax.plot(lambdalar, katsayilar) 
ax.set_xscale('log') 

plt.xlabel('Lambda(Alpha) Değerleri')
plt.ylabel('Katsayılar/Ağırlıklar')
plt.title('Düzenlileştirmenin Bir Fonksiyonu Olarak Ridge Katsayıları');

In [ ]:
y_pred = ridge_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
lambdalar = 10**np.linspace(10,-2,100)*0.5 

In [ ]:
from sklearn.linear_model import RidgeCV
ridge_cv = RidgeCV(alphas = lambdalar, 
                   scoring = "neg_mean_squared_error")

In [ ]:
ridge_cv.fit(X_train, y_train)

In [ ]:
ridge_cv.alpha_

In [ ]:
ridge_tuned = Ridge(alpha = ridge_cv.alpha_).fit(X_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(y_test, ridge_tuned.predict(X_test)))

In [ ]:
df.head()

In [ ]:
X = df.drop(['smoker'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_model = Lasso(alpha = 0.1).fit(X_train, y_train)

In [ ]:
lasso = Lasso()
lambdalar = 10**np.linspace(10,-2,100)*0.5 
katsayilar = []

for i in lambdalar:
    lasso.set_params(alpha=i)
    lasso.fit(X_train, y_train)
    katsayilar.append(lasso.coef_)
    
ax = plt.gca()
ax.plot(lambdalar*2, katsayilar)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
y_pred = lasso_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
df.head()

In [ ]:
X = df.drop(['smoker'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
enet_model = ElasticNet().fit(X_train, y_train)

In [ ]:
y_pred = enet_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
enet_cv_model = ElasticNetCV(cv = 10, random_state = 0).fit(X_train, y_train)

In [ ]:
enet_tuned = ElasticNet(alpha = enet_cv_model.alpha_).fit(X_train,y_train)

In [ ]:
y_pred = enet_tuned.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))